In [1]:
import csv
import os
import pandas as pd
from configparser import ConfigParser
from mysql.connector import MySQLConnection, Error

Utils

In [2]:
def read_config(config_file = 'config.ini', section = 'mysql'):
    """
    Read the configuration file config_file with the given section.
    If successful, return the configuration as a dictionary,
    else raise an exception.
    """
    parser = ConfigParser()
    
    # Does the configuration file exist?
    if os.path.isfile(config_file):
        parser.read(config_file)
    else:
        raise Exception(f"Configuration file '{config_file}' "
                        "doesn't exist.")
    
    config = {}
    
    if parser.has_section(section):
        # Parse the configuration file.
        items = parser.items(section)
        
        # Construct the parameter dictionary.
        for item in items:
            config[item[0]] = item[1]
            
    else:
        raise Exception(f'Section [{section}] missing ' + \
                        f'in config file {config_file}')
    
    return config

In [3]:
def make_connection(config_file = 'config.ini', section = 'mysql'):
    """
    Make a database connection with the configuration file config_file
    with the given section. If successful, return the connection,
    else raise an exception.
    """
    try:
        db_config = read_config(config_file, section)
        conn = MySQLConnection(**db_config)

        if conn.is_connected():
            return conn

    except Error as e:
        raise Exception(f'Connection failed: {e}')


In [4]:
def dataframe_query(conn, sql):
    """
    Use the database connection conn to execute
    the SQL code. Return the resulting row count
    and the rows as a dataframe or (0, None) 
    if there were no rows. If the query failed,
    raise an exception.
    """
    try:
        cursor = conn.cursor()
        cursor.execute(sql)

        rows  = cursor.fetchall()
        count = cursor.rowcount

        if count > 0:

            # Get the names of the columns.
            columns = cursor.description
            column_names = [column_info[0] 
                            for column_info in columns]

            # Return the query results in a dataframe.
            df = DataFrame(rows)
            df.columns = column_names
            cursor.close()
            return count, df

        else:
            cursor.close()
            return 0, None
        
    except Error as e:
        raise Exception(f'Query failed: {e}')

In [5]:
#testdf = pd.read_csv('movies_metadata.csv', usecols = ['original_language'])

In [6]:
#testdf['original_language'].unique()

Connecting

In [7]:
conn = make_connection(config_file='movies.ini')
cursor = conn.cursor()

Creating Tables

In [8]:
cursor.execute('DROP TABLE IF EXISTS links_small')

sql = ( """
        CREATE TABLE links_small
        (
            movieId    int,
            imdbId   int,
            tmdbId    int,
            PRIMARY KEY(movieId)
        )
        """
      )

cursor.execute(sql)

In [9]:
# Since userId has duplicates, movieId may contain duplicates, we cannot create a primary key without changing things.
# Temporarily added an auto increment to set as a primary key
cursor.execute('DROP TABLE IF EXISTS ratings')

sql = ( """
        CREATE TABLE ratings
        (
            ID_column int AUTO_INCREMENT,
            userId    int,
            movieId   int,
            rating    double,
            timestamp double,
            PRIMARY KEY(ID_column)
        )
        """
      )

cursor.execute(sql)

In [10]:
# Could use.
cursor.execute('DROP TABLE IF EXISTS movies_metadata')
sql = ( """
        CREATE TABLE movies_metadata
        (
            budget int,
            id int,
            imdb_id varchar(255),
            original_language char(255),
            original_title varchar(255),
            PRIMARY KEY(id)
        )
        """
      )
cursor.execute(sql)

Insert Commands

In [11]:
sql_links_small = ("""
                   INSERT INTO links_small
                   VALUES (%s,%s,%s)
                   """
                  )
sql_ratings = ("""
                INSERT INTO ratings (userId, movieId, rating, timestamp)
                VALUES (%s,%s,%s,%s)
                """
              )
sql_metadata = ("""
                INSERT INTO movies_metadata
                VALUES (%s,%s,%s,%s,%s)
                """
               )

Testing for Dupes

In [12]:
# Testing to see if ratings has duplicate movies for one to many
dupe1 = pd.read_csv('ratings.csv')
dupe1[dupe1.duplicated('movieId', keep=False)==True]

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
...,...,...,...,...
26024284,270896,58559,5.0,1257031564
26024285,270896,60069,5.0,1257032032
26024286,270896,63082,4.5,1257031764
26024287,270896,64957,4.5,1257033990


In [13]:
lis = []
id = 110
i=1
while len(lis) < 10:
    if dupe1['movieId'][i] == id:
        lis.append((dupe1['userId'][i], dupe1['movieId'][i], dupe1['rating'][i]))
    i+=1
lis

[(11, 110, 3.5),
 (22, 110, 5.0),
 (24, 110, 5.0),
 (29, 110, 3.0),
 (30, 110, 5.0),
 (33, 110, 3.0),
 (34, 110, 5.0),
 (49, 110, 4.0),
 (56, 110, 4.0),
 (58, 110, 5.0)]

Inserting the Data into the Tables

In [14]:
count = 0
first = True
with open('links_small.csv', newline='') as csv_file:
    data = csv.reader(csv_file,delimiter = ',', quotechar='"')
    for row in data:
        if not first:
            count+=1
            if count > 100:
                break
            cursor.execute(sql_links_small, row)
        first = False
conn.commit()

In [15]:
count = 0
first = True
with open('ratings.csv', newline='') as csv_file:
    data = csv.reader(csv_file,delimiter = ',', quotechar='"')
    for row in data:
        if not first:
            count+=1
            if count > 100:
                break
            cursor.execute(sql_ratings, row)
        first = False
conn.commit()

In [16]:
# Only want speciic columns?; Could -perform ETL and convert the language to its full name.
count = 0
first = True
testLis = [2, 5,6,7,8]
with open('movies_metadata.csv', newline='', encoding="utf-8") as csv_file:
    data = csv.reader(csv_file,delimiter = ',', quotechar='"')
    for row in data:
        if not first:
            test = [row[i] for i in testLis]
            count+=1
            if count > 100:
                break
            cursor.execute(sql_metadata, test)
        first = False
conn.commit()

Show the Tables in a DataFrame

In [20]:
select_links_small = ("""
                   SELECT * FROM links_small LIMIT 25
                   """
                  )
cursor.execute(select_links_small)
df1 = pd.DataFrame(cursor.fetchall())
df1
conn.commit()

In [18]:
select_ratings = ("""
                   SELECT * FROM ratings LIMIT 25
                   """
                  )
cursor.execute(select_ratings)
df2 = pd.DataFrame(cursor.fetchall())
df2
conn.commit()

In [19]:
select_metadata = ("""
                   SELECT * FROM movies_metadata LIMIT 25
                   """
                  )
cursor.execute(select_metadata)
df3 = pd.DataFrame(cursor.fetchall())
df3
conn.commit()

One to One, One to Many, Many to Many

In [18]:
# one to one
oto1 = (
        """
        SELECT m.id, m.original_title from movies_metadata as m, links_small as l
        WHERE m.id = l.movieId
        
        """
)
cursor.execute(oto1)
results = cursor.fetchall()
print(results)
conn.commit()

[(5, 'Four Rooms'), (63, 'Twelve Monkeys')]


In [20]:
# one to many
# This one might not work within the first 100 in the table.
otm1 = ("""
        SELECT l.movieId, r.rating from links_small as l, ratings as r
        WHERE l.movieId = r.movieId
        """)
cursor.execute(otm1)
results = cursor.fetchall()
print(results)
conn.commit()

[(5, 3.0), (25, 3.0), (32, 2.0), (58, 3.0), (64, 4.0), (79, 4.0)]


In [ ]:
# many to many

In [ ]:
cursor.close()
conn.close()